In [ ]:
pip install -e .

In [ ]:
# Import packages
import pandas as pd
from gsscoder_python import recode_gss

In [8]:
# Load the datasets
old_series = pd.read_parquet('/Users/user1/Downloads/origin_destination_2002_to_2020.parquet')
new_series = pd.read_csv('/Users/user1/Downloads/detailedestimates2022on2021and2023las.csv')
sample_df = pd.read_csv(
    's3://dpa-population-projection-data/population_mid_year_estimates/ons_data/cleaned_data_combined/cleaned_data_combined.csv',
    nrows=100000
)



In [9]:
def convert_age_data(df):
    
    # Rename key columns
    df = df.rename(columns={"outla": "gss_out", "inla": "gss_in", "Sex": "sex", "Year": "year"})

    # Melt age columns
    long_df = df.melt(
        id_vars=["gss_out", "gss_in", "sex", "year"],
        var_name="age",
        value_name="value"
    )

    # Clean up age and sex
    long_df["age"] = long_df["age"].str.replace("Age_", "", regex=False).astype(float)
    long_df["sex"] = long_df["sex"].map({"F": "female", "M": "male"})
    long_df["year"] = long_df["year"].astype(float)

    # Remove zero or missing values if desired
    long_df = long_df[long_df["value"] != 0]

    return long_df.reset_index(drop=True)

In [10]:
new_series_converted = convert_age_data(new_series)

In [11]:
new_series_converted.head()

,gss_out,gss_in,sex,year,age,value
0,E06000001,E06000004,female,2022.0,0.0,0.8027
1,E06000001,E06000004,male,2022.0,0.0,1.4287
2,E06000001,E06000047,female,2022.0,0.0,2.6226
3,E06000001,E06000047,male,2022.0,0.0,1.8014
4,E06000001,E06000054,female,2022.0,0.0,0.5747


In [12]:
geog_yr_old_series = 2021
geog_yr_new_series = 2023
start_yr_new_series = 2012

In [13]:
old_series

,gss_out,gss_in,year,sex,age,value
0,E06000001,E06000002,2002.0,female,7.0,1.1805
1,E06000001,E06000002,2002.0,female,8.0,1.1805
2,E06000001,E06000002,2002.0,female,9.0,1.1805
3,E06000001,E06000002,2002.0,female,11.0,2.3610
4,E06000001,E06000002,2002.0,female,12.0,1.1805
...,...,...,...,...,...,...
31196466,W06000024,W06000023,2020.0,male,34.0,1.1089
31196467,W06000024,W06000023,2020.0,male,35.0,1.1565
31196468,W06000024,W06000023,2020.0,male,43.0,1.1501
31196469,W06000024,W06000023,2020.0,male,66.0,1.1905


In [14]:
recoded_old_series = recode_gss(old_series,
                col_code = "gss_in",
               col_data = "value",
               recode_from_year = geog_yr_old_series,
               recode_to_year = geog_yr_new_series),